In [15]:
import pandas as pd

In [18]:
df = pd.read_csv('train.csv')
# df = pd.read_csv('train.csv',error_bad_lines= False, engine='pyhton')

In [19]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [20]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [39]:
## Drop nan values
df=df.dropna()

In [40]:
#Get the independent features
X=df.drop('label', axis=1)

In [41]:
##Get the dependent features
y=df['label']


In [42]:
X.shape

(18285, 4)

In [43]:
import tensorflow as tf

In [44]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [45]:
##vocublary size
voc_size=5000

ONEHOT Representation

In [46]:
messages = X.copy()

In [47]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [48]:
messages.reset_index(inplace=True)

In [49]:
import nltk
import re
from nltk.corpus import stopwords

In [50]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [51]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', str(messages['title'][i]))
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [52]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [53]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[120, 4562, 4856, 2200, 1080, 3994, 1158, 3358, 513, 3600],
 [4258, 2698, 4251, 236, 3321, 4588, 651],
 [799, 1867, 4222, 3811],
 [4378, 4604, 4445, 167, 3410, 2652],
 [2802, 3321, 1485, 4305, 4093, 147, 3321, 3940, 1789, 1747],
 [4319,
  1270,
  1715,
  946,
  3359,
  1492,
  2763,
  1557,
  4000,
  3385,
  3910,
  1448,
  804,
  1387,
  651],
 [4824, 3383, 3566, 1338, 2362, 2424, 1018, 675, 3876, 602, 2481],
 [221, 2184, 822, 1876, 4478, 1496, 1492, 1269, 3876, 602, 2481],
 [2007, 2024, 3357, 3406, 4442, 619, 3235, 4794, 1492, 2497],
 [1240, 1779, 2171, 2850, 1453, 2412, 1093, 1722],
 [505, 4540, 698, 3144, 1238, 492, 1655, 3289, 3534, 2669, 4472],
 [167, 1746, 1080, 619, 1492, 4478],
 [1298, 827, 1712, 880, 3747, 2193, 3770, 4667, 3915],
 [3361, 4639, 3289, 24, 1048, 2507, 1923, 3876, 602, 2481],
 [703, 2938, 1932, 1474, 3130, 3876, 602, 2481],
 [2181, 692, 3054, 4172, 1839, 2080, 4325, 74, 3260, 4345],
 [2495, 2031, 2698],
 [4961, 3229, 1983, 1816, 1492, 859, 3633, 651],
 [2966, 1

In [54]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [55]:
onehot_repr[1]

[4258, 2698, 4251, 236, 3321, 4588, 651]

Embedding Representation

In [56]:
sent_length= 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3358  513 3600]
 [   0    0    0 ... 3321 4588  651]
 [   0    0    0 ... 1867 4222 3811]
 ...
 [   0    0    0 ... 3876  602 2481]
 [   0    0    0 ... 4249 3799  844]
 [   0    0    0 ... 2927 2909 4594]]


In [57]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 4258, 2698, 4251,  236, 3321, 4588,  651], dtype=int32)

In [58]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [59]:
len(embedded_docs),y.shape

(18285, (18285,))

In [62]:
import numpy as np
X_final=np.array(embedded_docs)
y_final = np.array(y)

In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [74]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 40ms/step - loss: 0.3399 - accuracy: 0.8390 - val_loss: 0.1953 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 11s 59ms/step - loss: 0.1490 - accuracy: 0.9420 - val_loss: 0.2177 - val_accuracy: 0.9208
Epoch 3/10
192/192 [==============================] - 9s 49ms/step - loss: 0.1077 - accuracy: 0.9599 - val_loss: 0.2082 - val_accuracy: 0.9180
Epoch 4/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0798 - accuracy: 0.9704 - val_loss: 0.2527 - val_accuracy: 0.9148
Epoch 5/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0575 - accuracy: 0.9789 - val_loss: 0.2719 - val_accuracy: 0.9087
Epoch 6/10
192/192 [==============================] - 7s 37ms/step - loss: 0.0393 - accuracy: 0.9862 - val_loss: 0.3473 - val_accuracy: 0.9115
Epoch 7/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0331 - accuracy: 0.9890 - val_loss: 0.3569 - val_accuracy: 0.90

# Adding Dropout

In [73]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Performance Matrix and accuracy

In [75]:
y_pred=model.predict(X_test)


189/189 [==============================] - 2s 7ms/step


In [76]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [77]:
from sklearn.metrics import confusion_matrix

In [78]:
confusion_matrix(y_test,y_pred)

array([[3148,  271],
       [ 329, 2287]])

In [79]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9005799502899752

In [80]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      3419
           1       0.89      0.87      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

